# Socket Communication Network

This notebook is an implementation of a simple UDP socket server using *CoppeliaSim's* threaded childscripts as clients. 

The goal is to implement and analyse, the characteristics of the User Datagram Protocol for communication between the client and the server. For this:
1. Associate threaded childscripts to *CoppeliaSim's* Dummies;
2. Code the client side on these childscripts;
3. Send messages each sensing callback is done to the server;
4. Check Real Time Factor, Latency and Sending Rate. 

---

In [1]:
# Importing modules...
import numpy as np
import socket		

# Creating the Server's UDP Socket

The information bellow is taken from the GeeksForGeeks's website and is available [here](https://www.geeksforgeeks.org/differences-between-tcp-and-udp/).

## The User Datagram Protocol

User Datagram Protocol (UDP) is a Transport Layer Protocol. UDP is a part of the Internet Protocol suite, referred to as the UDP/IP suite. Unlike the Transmission Control Protocol (TCP), it is an unreliable and connectionless protocol. 

Therefore, there is no need to establish a connection before data transfer. The UDP helps to establish **low-latency** and **loss-tolerating** connections establish over the network. The UDP enables process-to-process communication - an event based communication.

- Used for simple request-response communication when the size of data is less and hence there is lesser concern about flow and error control.
- It is a suitable protocol for multicasting as UDP supports packet switching.
- UDP is used for some routing update protocols like RIP (Routing Information Protocol).
- Normally used for real-time applications which can not tolerate uneven delays between sections of a received message.

### Advantages of UDP
- It does not require any connection for sending or receiving data;
- Broadcast and Multicast are available in UDP;
- UDP can operate on a large range of networks;
- UDP has live and real-time data;
- UDP can deliver data if all the components of the data are not complete.

### Disadvantages of UDP
- We can not have any way to acknowledge the successful transfer of data;
- UDP cannot have the mechanism to track the sequence of data;
- UDP is connectionless, and due to this, it is unreliable to transfer data;
- In case of a Collision, UDP packets are dropped by routers in comparison to TCP;
- UDP can drop packets in case of detection of errors.

---

In [2]:
print('> Creating server socket...')

# Try to create server socket
try: 
    server_socket = socket.socket(socket.AF_INET,    # Internet
                                  socket.SOCK_DGRAM) # UDP
    print('> Socket successfully created')
    
except socket.error as err: 
    print(f'> Socket creation failed with error {err}\n')
    print('> Quitting server...')
    exit()

UDP_IP = '127.0.0.1' # Server IP
UDP_PORT = 8888      # UDP Port
buffer_size = 1024   # In bytes

server_socket.bind((UDP_IP, UDP_PORT))
print(f'> Server bound to port {UDP_PORT}')

> Creating server socket...
> Socket successfully created
> Server bound to port 8888


# Identifying Clients Addresses

After starting the server, it will wait for clients to send messages to it with their ID number and save their addresses (IP and Port). This routine waits until all clients identify themselves. 

---

In [3]:
n_clients = 4
address_list = {}

print('> Waiting for clients...')

# Address lookup 
while len(address_list.keys()) < n_clients: # Until all clients are identified
    message_bytes, address = server_socket.recvfrom(buffer_size)

    try:
        ID = int(message_bytes.decode()) # Decode message

    except: # Invalid message for decoding
        continue # Look for another message

    address_list[address] = ID

    print(f'\tClient {ID} Connected')

print('> All clients connected!')

> Waiting for clients...
	Client 0 Connected
	Client 1 Connected
	Client 2 Connected
	Client 3 Connected
> All clients connected!


# Receiving Client Messages

With all the clients identified, the server will now start receiving the desired messages sent by them. For avoiding the server to run indefinetely, a timeout will be set to close the socket after the timeout. 

In this simplified case, clients will only send the Simulation Timestamp they are being triggered to send the message.  

In addition, ot avoid output pollution, the server will only receive until a target timestamp is reached.

---

In [4]:
timeout = 10 # In seconds
server_socket.settimeout(timeout) # Set server timeout
print(f'> Server timeout set to {timeout} seconds\n')

PTS = 0.0 # Presentation Time Stamp
target_PTS  = 1.0 # Only receive messages until this timestamp 

while PTS < target_PTS:  
    # Wait for message - Event guided!
    try:
        message_bytes, address = server_socket.recvfrom(buffer_size)

    except socket.timeout as err:
        print('\n> Server timed out')
        break # Close loop due to timeout
    
    # Decode message 
    message = np.frombuffer(message_bytes, dtype=np.float64)

    detected_blobs = message[:-1].reshape(-1, 2) # All but last element are the blob coordinates
    PTS = message[-1] # Last element contains PTS

    print(f'> Received message from Client {address_list[address]} ({address[0]}, {address[1]}):')
    print(f'\tDetected Blobs - {PTS :.3f} s')
    print('\t' + str(detected_blobs).replace('\n', '\n\t'))


print('\nConnection Endend.')

> Server timeout set to 10 seconds

> Received message from Client 3 (127.0.0.1, 60602):
	Detected Blobs - 0.000 s
	[[359.30000019 246.30000019]]
> Received message from Client 0 (127.0.0.1, 53058):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 1 (127.0.0.1, 58873):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 2 (127.0.0.1, 58874):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 3 (127.0.0.1, 60602):
	Detected Blobs - 0.050 s
	[[359.30000019 243.30000019]]
> Received message from Client 0 (127.0.0.1, 53058):
	Detected Blobs - 0.100 s
	[[359.14938927 240.61400747]]
> Received message from Client 1 (127.0.0.1, 58873):
	Detected Blobs - 0.100 s
	[[359.30000019 240.30000019]]
> Received message from Client 2 (127.0.0.1, 58874):
	Detected Blobs - 0.100 s
	[[359.30000019 240.30000019]]
> Received message from Client 3 (127.0.0.1, 60602):
	Detected Blobs - 0.100 s
	[[35